## Quickstart

- https://mlflow.org/docs/latest/getting-started/intro-quickstart/index.html

- The TL;DR version of MLFlow tracking, covers only essentials
- This assumes you already did section 1, which is to start an mlflow server

### Setting up connection to server

In [5]:
import mlflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("MLflow Tracking Quickstart")

2024/12/23 19:35:41 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Tracking Quickstart' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/370347002317159941', creation_time=1734953741840, experiment_id='370347002317159941', last_update_time=1734953741840, lifecycle_stage='active', name='MLflow Tracking Quickstart', tags={}>

### Train dummy model

In [2]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

/Users/yongjian.tek/Desktop/data_science/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [6]:
# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

### Log dummy model 

In [7]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

2024/12/23 19:36:17 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'tracking-quickstart'.
2024/12/23 19:36:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1


🏃 View run fearless-carp-660 at: http://127.0.0.1:5000/#/experiments/370347002317159941/runs/4dadc71adcb243c28003913393440d01
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/370347002317159941


Created version '1' of model 'tracking-quickstart'.


### Load model for inference

In [8]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


## MLFlow Tracking In-Depth

### Start mlflow server

- Start server if you haven't already
    - `uv tool run mlflow server --host 127.0.0.1 --port 8080`

### Configure tracking client 

In [8]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

In [9]:
client = MlflowClient(tracking_uri="http://127.0.0.1:5000")

### Experiments

- Every project should be logged within an experiment, and you can set this via the `mlflow.set_experiment("...")` method
- However, even if you should forget to set an experiment, mlflow will automatically create a "Default" that tracks all experiments

In [10]:
all_experiments = client.search_experiments()
[x for x in all_experiments]

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1734955750065, experiment_id='0', last_update_time=1734955750065, lifecycle_stage='active', name='Default', tags={}>]

In [11]:
default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


### Tagging Experiments

- Nevertheless, it is highly recommended to set experiments rather than relying on the `Default` safety net
    - Because experiments allow you to add metadata and organise multiple runs such that multiple models under the safe project can be logged together

- Imagine a forecasting project for a grocery store; and we want demand forecasting for each individual item; logging each individual item as a model will help us track performance more precisely
    ```
    - Demand forecasting
        - Dairy
            - Cheese
                - Parmesan
                - Cheddar
            - Milk
                - Whole
                - 2%
        - Produce
            - Fruit
                - Apples
                - Cherries
            - Vegetables
                - Carrots
    ```

- Roughly, the hierachy or organisation is:
    - Tags (i.e. `Produce`, `Dairy`)
    - Experiments (i.e. `Cheese`, `Milk`)
    - Runs (individual model training runs)

In [12]:
# Provide an Experiment description that will appear in the UI
experiment_description = (
    "This is the grocery forecasting project. "
    "This experiment contains the produce models for apples."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "grocery-forecasting",
    "store_dept": "produce",
    "team": "stores-ml",
    "project_quarter": "Q3-2023",
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
produce_apples_experiment = client.create_experiment(
    name="Apple_Models", tags=experiment_tags
)

### Searching Experiments

In [15]:
apples_experiment = client.search_experiments(
    filter_string="tags.`project_name` = 'grocery-forecasting'"
)
vars(apples_experiment[0])

{'_experiment_id': '699058987587037737',
 '_name': 'Apple_Models',
 '_artifact_location': 'mlflow-artifacts:/699058987587037737',
 '_lifecycle_stage': 'active',
 '_tags': {'project_name': 'grocery-forecasting',
  'project_quarter': 'Q3-2023',
  'mlflow.note.content': 'This is the grocery forecasting project. This experiment contains the produce models for apples.',
  'store_dept': 'produce',
  'team': 'stores-ml'},
 '_creation_time': 1734955769962,
 '_last_update_time': 1734955769962}

### Generate Synthetic Dataset

In [17]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


def generate_apple_sales_data_with_promo_adjustment(
    base_demand: int = 1000, n_rows: int = 5000
):
    """
    Generates a synthetic dataset for predicting apple sales demand with seasonality
    and inflation.

    This function creates a pandas DataFrame with features relevant to apple sales.
    The features include date, average_temperature, rainfall, weekend flag, holiday flag,
    promotional flag, price_per_kg, and the previous day's demand. The target variable,
    'demand', is generated based on a combination of these features with some added noise.

    Args:
        base_demand (int, optional): Base demand for apples. Defaults to 1000.
        n_rows (int, optional): Number of rows (days) of data to generate. Defaults to 5000.

    Returns:
        pd.DataFrame: DataFrame with features and target variable for apple sales prediction.

    Example:
        >>> df = generate_apple_sales_data_with_seasonality(base_demand=1200, n_rows=6000)
        >>> df.head()
    """

    # Set seed for reproducibility
    np.random.seed(9999)

    # Create date range
    dates = [datetime.now() - timedelta(days=i) for i in range(n_rows)]
    dates.reverse()

    # Generate features
    df = pd.DataFrame(
        {
            "date": dates,
            "average_temperature": np.random.uniform(10, 35, n_rows),
            "rainfall": np.random.exponential(5, n_rows),
            "weekend": [(date.weekday() >= 5) * 1 for date in dates],
            "holiday": np.random.choice([0, 1], n_rows, p=[0.97, 0.03]),
            "price_per_kg": np.random.uniform(0.5, 3, n_rows),
            "month": [date.month for date in dates],
        }
    )

    # Introduce inflation over time (years)
    df["inflation_multiplier"] = (
        1 + (df["date"].dt.year - df["date"].dt.year.min()) * 0.03
    )

    # Incorporate seasonality due to apple harvests
    df["harvest_effect"] = np.sin(2 * np.pi * (df["month"] - 3) / 12) + np.sin(
        2 * np.pi * (df["month"] - 9) / 12
    )

    # Modify the price_per_kg based on harvest effect
    df["price_per_kg"] = df["price_per_kg"] - df["harvest_effect"] * 0.5

    # Adjust promo periods to coincide with periods lagging peak harvest by 1 month
    peak_months = [4, 10]  # months following the peak availability
    df["promo"] = np.where(
        df["month"].isin(peak_months),
        1,
        np.random.choice([0, 1], n_rows, p=[0.85, 0.15]),
    )

    # Generate target variable based on features
    base_price_effect = -df["price_per_kg"] * 50
    seasonality_effect = df["harvest_effect"] * 50
    promo_effect = df["promo"] * 200

    df["demand"] = (
        base_demand
        + base_price_effect
        + seasonality_effect
        + promo_effect
        + df["weekend"] * 300
        + np.random.normal(0, 50, n_rows)
    ) * df[
        "inflation_multiplier"
    ]  # adding random noise

    # Add previous day's demand
    df["previous_days_demand"] = df["demand"].shift(1)
    df["previous_days_demand"].fillna(
        method="bfill", inplace=True
    )  # fill the first row

    # Drop temporary columns
    df.drop(columns=["inflation_multiplier", "harvest_effect", "month"], inplace=True)

    return df

In [18]:
data = generate_apple_sales_data_with_promo_adjustment(base_demand=1_000, n_rows=1_000)
data[-20:]

/var/folders/hs/wbbxdff91wj8m8np2cv6jnnw0000gq/T/ipykernel_37008/3859398416.py:89: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["previous_days_demand"].fillna(
/var/folders/hs/wbbxdff91wj8m8np2cv6jnnw0000gq/T/ipykernel_37008/3859398416.py:89: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["previous_days_demand"].fillna(


,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand
980,2024-12-04 20:13:10.889331,34.130183,1.454065,0,0,1.449177,0,971.802447,1001.085782
981,2024-12-05 20:13:10.889330,32.353643,9.462859,0,0,2.856503,0,818.951553,971.802447
982,2024-12-06 20:13:10.889329,18.816833,0.391470,0,0,1.326429,0,963.352029,818.951553
983,2024-12-07 20:13:10.889329,34.533012,2.120477,1,0,0.970131,0,1357.385504,963.352029
984,2024-12-08 20:13:10.889328,23.057202,2.365705,1,0,1.049931,0,1309.427049,1357.385504
985,2024-12-09 20:13:10.889328,34.810165,3.089005,0,0,2.035149,0,974.971149,1309.427049
986,2024-12-10 20:13:10.889327,29.208905,3.673292,0,0,2.518098,0,1056.249547,974.971149
987,2024-12-11 20:13:10.889326,16.428676,4.077782,0,0,1.268979,0,1063.118915,1056.249547
988,2024-12-12 20:13:10.889326,32.067512,2.734454,0,0,0.762317,0,1040.492007,1063.118915
989,2024-12-13 20:13:10.889325,31.938203,13.883486,0,0,1.153301,0,967.040470,1040.492007


### Log run with synthetic data

In [19]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [22]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [23]:
# Sets the current active experiment to the "Apple_Models" experiment and
# returns the Experiment metadata
apple_experiment = mlflow.set_experiment("Apple_Models")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "apples_rf_test"

# Define an artifact path that the model will be saved to.
artifact_path = "rf_apples"

In [24]:
# Split the data into features and target and drop irrelevant date field and target field
X = data.drop(columns=["date", "demand"])
y = data["demand"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    "n_estimators": 100,
    "max_depth": 6,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "bootstrap": True,
    "oob_score": False,
    "random_state": 888,
}

# Train the RandomForestRegressor
rf = RandomForestRegressor(**params)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf.predict(X_val)

# Calculate error metrics
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

# Assemble the metrics we're going to write into a collection
metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(
        sk_model=rf, input_example=X_val, artifact_path=artifact_path
    )

/Users/yongjian.tek/Desktop/data_science/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/12/23 20:43:00 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run apples_rf_test at: http://127.0.0.1:5000/#/experiments/699058987587037737/runs/206dbadea21f4041bb47c4f4d9bcdb67
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/699058987587037737
